In [10]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.80


In [11]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config(path="aml_config/config.json")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: D:\Projects\LEGO Vision\Notebooks\Azure Machine Learning Service\aml_config\config.json
Workspace name: LEGO-Vision
Azure region: westeurope
Subscription id: 2f5c43ac-ac34-401d-bead-2dc43ffdc4b1
Resource group: LEGO-Vision


In [30]:
from azureml.core.compute import ComputeTarget, BatchAiCompute, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "LEGOVISION"

try:
    compute_target_one = AmlCompute(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           autoscale_enabled=True,
                                                           min_nodes=1, 
                                                           max_nodes=3)

    # create the cluster
    compute_target_one = AmlCompute.create(ws, cluster_name, compute_config)

    compute_target_one.wait_for_completion(show_output=True)

    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target_one.status.serialize())

Creating a new compute target...
Creating.
AmlCompute wait for completion finished
There were errors reported from AmlCompute:
[{'code': 'AllocationFailed', 'message': 'Desired number of dedicated nodes could not be allocated as they are not available', 'details': None}]
{'allocationState': 'steady', 'allocationStateTransitionTime': '2018-11-23T15:17:13.854000+00:00', 'creationTime': '2018-11-23T15:17:04.089000+00:00', 'currentNodeCount': 0, 'errors': [{'code': 'AllocationFailed', 'message': 'Desired number of dedicated nodes could not be allocated as they are not available', 'details': None}], 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'succeeded', 'provisioningStateTransitionTime': '2018-11-23T15:17:13.386000+00:00', 'scaleSettings': {'manual': None, 'autoScale': {'maximumNodeCount': 3, 'minimumNodeCount': 1, 'initialNodeCount': 1}}, 'vmPriority': 'dedicated', 'vmSize': '

In [19]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureFile legovision1529597820 azureml-filestore-f0102bff-6d0a-47ad-b50c-4709fd2f206a


In [20]:
annotated_data = ds.path('Object Detection')
print(annotated_data)

$AZUREML_DATAREFERENCE_3e2b34d7f366477f9eecec8b60a85f87


In [21]:
import shutil
project_folder = r'./aml_directories/faster-rcnn-aml'
shutil.rmtree(project_folder)
shutil.copytree('..\..\Detection\CNN\FasterRCNN', project_folder)

'./faster-rcnn-aml'

In [22]:
from azureml.core import Experiment

experiment_name = 'Faster_R-CNN'
experiment = Experiment(ws, name=experiment_name)

In [23]:
from azureml.train.estimator import *

script_params = {
    '--base_directory': annotated_data.as_mount(),
    '--data_set_name': "LEGO Vision 500s"
}

estimator = Estimator(source_directory=project_folder,
                      compute_target=compute_target_one,
                      entry_script='run_faster_rcnn.py',
                      script_params=script_params,
                      pip_packages=['cntk-gpu==2.6'],
                      custom_docker_base_image='microsoft/mmlspark:gpu',
                      use_gpu=True)

In [24]:
run = experiment.submit(estimator)

In [28]:
from azureml.train.widgets import RunDetails
RunDetails(run).show()

ImportError: cannot import name '_run_history_service_client'